In [1]:
import numpy as np
from datetime import datetime

In [2]:
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [8]:
class LogicGate:
    
    def __init__(self, gate_name, i_node, h1_node, o_node, learning_rate):  
        self.name = gate_name
        
        self.W2 = np.random.rand(i_node, h1_node)             
        self.b2 = np.random.rand(h1_node)
        
        self.W3 = np.random.rand(h1_node, o_node) 
        self.b3 = np.random.rand(o_node)
                        
        self.learning_rate = learning_rate
        
        print(self.name + " object is created")

    def feed_forward(self):
        delta = 1e-7   
        
        Z2 = np.dot(self.xdata, self.W2) + self.b2
        A2 = sigmoid(Z2)
        
        Z3 = np.dot(A2, self.W3) + self.b3
        y = sigmoid(Z3)
        
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    def loss_val(self):
        delta = 1e-7   
    
        Z2 = np.dot(self.xdata, self.W2) + self.b2    
        A2 = sigmoid(Z2)    
    
        Z3 = np.dot(A2, self.W3) + self.b3    
        y = sigmoid(Z3)    
        
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
                       
    def train(self, xdata, tdata):
        self.xdata = xdata
        self.tdata = tdata
        
        f = lambda x : self.feed_forward()
      
        self.W2 -= self.learning_rate * numerical_derivative( f, self.W2 )
        self.b2 -= self.learning_rate * numerical_derivative( f, self.b2 )

        self.W3 -= self.learning_rate * numerical_derivative( f, self.W3 )
        self.b3 -= self.learning_rate * numerical_derivative( f, self.b3 )
                       
    def predict(self, x):
        z2 = np.dot(x, self.W2) + self.b2
        a2 = sigmoid(z2)
    
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy1(self, test_xdata, test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(True)
            else:
                not_matched_list.append(False)
        
        accuracy_result = len(matched_list) / len(test_xdata)
        print("\nAccuracy =>", accuracy_result)
        
        return matched_list, not_matched_list
    
    def accuracy2(self, test_data):
        matched_list = []
        not_matched_list = []
        
        test_xdata = test_data[:, 0:-1]
        test_tdata = test_data[:, -1]
        
        for index in range(len(test_xdata)):            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(True)
            else:
                not_matched_list.append(False)
        
        accuracy_result = len(matched_list) / len(test_xdata)
        print("\nAccuracy =>", accuracy_result)
        
        return matched_list, not_matched_list

## AND

In [12]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])
i_node = 2
h1_node = 1
o_node = 1
learning_rate = 1e-1
epochs = 5000

AND_obj = LogicGate("AND_GATE", i_node, h1_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for count in  range(epochs):
    for index in range(len(xdata)):
        AND_obj.train(xdata[index], tdata[index])
    
    if (count % 500 == 0):
        print("count =", count, "loss value =", AND_obj.loss_val())
                
end_time = datetime.now()

print("Elapsed Time =>", end_time - start_time)

AND_GATE object is created
count = 0 loss value = 0.4050528398975581
count = 500 loss value = 0.1517867448741649
count = 1000 loss value = 0.04077284257082444
count = 1500 loss value = 0.022264564948683337
count = 2000 loss value = 0.015106195423043065
count = 2500 loss value = 0.011367221528904303
count = 3000 loss value = 0.009085499938761538
count = 3500 loss value = 0.007553618619753644
count = 4000 loss value = 0.006456540803059106
count = 4500 loss value = 0.005633356569651226
Elapsed Time => 0:00:04.603697


In [13]:
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

for data in test_xdata:
    (real_val, logical_val) = AND_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = AND_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = AND_obj.accuracy2(test_data)

print("\n", accuracy_ret)

AND_GATE 

real_val = [0.00108795] , logical_val = 0
real_val = [0.0029829] , logical_val = 0
real_val = [0.00298511] , logical_val = 0
real_val = [0.99501768] , logical_val = 1

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


## OR

In [15]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ]).reshape(4, 2)
tdata = np.array([0, 1, 1, 1]).reshape(4, 1)
i_node = 2
h1_node = 1
o_node = 1
learning_rate = 1e-1
epochs = 5000

OR_obj = LogicGate("OR_GATE", i_node, h1_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for count in  range(epochs):
    for index in range(len(xdata)):
        OR_obj.train(xdata[index], tdata[index])
    
    if (count % 500 == 0):
        print("count =", count, "loss value =", OR_obj.loss_val())
                
end_time = datetime.now()

print("Elapsed Time =>", end_time - start_time)

OR_GATE object is created
count = 0 loss value = 0.33315243197072736
count = 500 loss value = 0.01357355314779132
count = 1000 loss value = 0.005365666952123143
count = 1500 loss value = 0.0033207372788836994
count = 2000 loss value = 0.002404723704560233
count = 2500 loss value = 0.0018861352614835665
count = 3000 loss value = 0.001552550804173204
count = 3500 loss value = 0.0013199348782984394
count = 4000 loss value = 0.0011484357594043072
count = 4500 loss value = 0.00101673361461488
Elapsed Time => 0:00:04.299537


In [16]:
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

for data in test_xdata:
    (real_val, logical_val) = OR_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = OR_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = OR_obj.accuracy2(test_data)

print("\n", accuracy_ret)

OR_GATE 

real_val = [0.00657692] , logical_val = 0
real_val = [0.99819086] , logical_val = 1
real_val = [0.99818758] , logical_val = 1
real_val = [0.99908774] , logical_val = 1

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


## NAND

In [17]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ]).reshape(4, 2)
tdata = np.array([1, 1, 1, 0]).reshape(4, 1)
i_node = 2
h1_node = 1
o_node = 1
learning_rate = 1e-1
epochs = 5000

NAND_obj = LogicGate("NAND_GATE", i_node, h1_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for count in  range(epochs):
    for index in range(len(xdata)):
        NAND_obj.train(xdata[index], tdata[index])
    
    if (count % 500 == 0):
        print("count =", count, "loss value =", NAND_obj.loss_val())
                
end_time = datetime.now()

print("Elapsed Time =>", end_time - start_time)

NAND_GATE object is created
count = 0 loss value = 0.9671119520407513
count = 500 loss value = 0.22142635339598715
count = 1000 loss value = 0.04616132623097817
count = 1500 loss value = 0.02388103523462765
count = 2000 loss value = 0.01585201846629439
count = 2500 loss value = 0.011789798651570151
count = 3000 loss value = 0.009355371216234195
count = 3500 loss value = 0.0077399871321911316
count = 4000 loss value = 0.006592526617457334
count = 4500 loss value = 0.005736713689179747
Elapsed Time => 0:00:04.352369


In [18]:
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

for data in test_xdata:
    (real_val, logical_val) = NAND_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = NAND_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = NAND_obj.accuracy2(test_data)

print("\n", accuracy_ret)

NAND_GATE 

real_val = [0.99889966] , logical_val = 1
real_val = [0.99696784] , logical_val = 1
real_val = [0.99696555] , logical_val = 1
real_val = [0.00506302] , logical_val = 0

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


## XOR

In [19]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])
i_node = 2
h1_node = 2
o_node = 1
learning_rate = 1e-1
epochs = 5000

XOR_obj = LogicGate("XOR_GATE", i_node, h1_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for count in  range(epochs):
    for index in range(len(xdata)):
        XOR_obj.train(xdata[index], tdata[index])
    
    if (count % 500 == 0):
        print("count =", count, "loss value =", XOR_obj.loss_val())
                
end_time = datetime.now()

print("Elapsed Time =>", end_time - start_time)

XOR_GATE object is created
count = 0 loss value = 1.3023628862498764
count = 500 loss value = 0.692955604549672
count = 1000 loss value = 0.6985059428312256
count = 1500 loss value = 0.7123825754945191
count = 2000 loss value = 0.8828649176883808
count = 2500 loss value = 0.7239519643703307
count = 3000 loss value = 0.10285674958382642
count = 3500 loss value = 0.0436643214726506
count = 4000 loss value = 0.026877895389279406
count = 4500 loss value = 0.01923047993025
Elapsed Time => 0:00:07.594704


In [20]:
print(XOR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

for data in test_xdata:
    (real_val, logical_val) = XOR_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = XOR_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = XOR_obj.accuracy2(test_data)

print("\n", accuracy_ret)

XOR_GATE 

real_val = [0.00852048] , logical_val = 0
real_val = [0.9904631] , logical_val = 1
real_val = [0.99042556] , logical_val = 1
real_val = [0.01480403] , logical_val = 0

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])
